In [ ]:
#Data Extraction using Beautiful Soup
def data_extraction():
    #Libraries For Data Extraction
    from urllib.request import urlopen
    from bs4 import BeautifulSoup
    import time
    import datetime
    from PyQt5.QtGui import QApplication #Fix imports. New imports as of latest update of PyQt
    from PyQt5.QtCore import QUrl
    from PyQt5.QtWebKit import QWebpage
    
    class Client(QWebPage):
        def __init__(self, url):
            self.app = QApplication(sys.argv)
            QWebPage.__init__(self)
            self.loadFinished.connect(self.on_page_load)
            self.mainFrame().load(QUrl(url))
            self.app.exec_()
            
    def on_page_load(self):
        self.app.quit()
        
    #specify the url
    url = 'http://mmdatraffic.interaksyon.com/line-view-edsa.php'
    client_response = Client(url)
    source = client_response.mainFrame().toHtml()

    # parse the html 
    soup = BeautifulSoup(source, 'lxml')

    #set time stamp.
    ts = time.time()
    timestamp = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d_%H_%M_%S')
    stringdata = "TIMESTAMP: "+ str(timestamp)+"\n"+"\n"

    #extract name of lines
    list_of_names_html=soup.find_all('div',{'class':'line-name'})
    list_of_names = []

    for children in list_of_names_html:
        grandchildren = children.findChild("p")
        temp = grandchildren.get_text(separator=' ')
        temp = temp.split(' ')
        list_of_names.append(temp[0]) #NOTE: fix string separation
    list_of_names.pop(0) 

    #extract southbound/northbound volume
    list_of_volume_html=soup.find_all('div',{'class':'line-status'})
    list_of_southbound = []
    list_of_northbound = []

    i=1
    for children in list_of_volume_html:
        temp = children.text
        temp = temp.split()

        if(temp[1]=="LIGHT"):
            temp[1]=0
        elif(temp[1]=="MODERATE"):
            temp[1]=1
        elif(temp[1]=="HEAVY"):
            temp[1]=2

        if(i%2 == 0):
            list_of_northbound.append(temp[1])
        else:
            list_of_southbound.append(temp[1])

        i=i+1

    #save data into one variable
    for i in range(len(list_of_names)):
        stringdata = stringdata + '{:>12}  {:>12}  {:>12}'.format(list_of_names[i], str(list_of_southbound[i]), str(list_of_northbound[i])) + "\n"
    
    #Save raw data into csv file
    filename = "rawdata_"+ str(timestamp)+".csv"
    file = open(filename,"a")
    file.write(stringdata)   
    file.close()
    
    #return list containing data lists
    list_of_lists = [list_of_names, list_of_southbound, list_of_northbound, timestamp]
    return list_of_lists

In [ ]:
# Visualization using Matplotlib
def visualize_overall(list_of_names, list_of_southbound, list_of_northbound,timestamp):
    #Libraries For Visualization and Analysis
    %matplotlib notebook
    import pandas as pd
    import numpy as np
    import seaborn as sns
    from matplotlib import pyplot as plt
    import matplotlib.animation as animation
    
    #Visualize overall EDSA traffic at certain timestamp
    roadnum = range(0,len(list_of_names))
    plt.scatter(roadnum, list_of_southbound, label="SOUTHBOUND", color="g",marker = "s")
    plt.scatter(roadnum, list_of_northbound, label = "NORTHBOUND", color = "m",marker = "|")

    plt.xlabel("Road")
    plt.ylabel("Volume")

    plt.title(label="EDSA TRAFFIC: "+timestamp)
    plt.legend(bbox_to_anchor=(0, 1), loc='lower center', ncol=1)
    plt.savefig("EDSA TRAFFIC "+timestamp+".png") #finally, save the figure  

In [ ]:
#Define constants here
global target_runs
global individual

target_runs = 18 #3 hours
individual = 22 #Pioneer-Boni

#Libraries For Scheduling
import schedule
import time
import datetime
import os, os.path 
from sys import exit

#Libraries for visualization (make global imports later)
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation

#Libraries for analysis
from scipy.interpolate import *
    
southbound_data = [] #append southbound data of individual here.
northbound_data = [] #append northbound data of individual here. 
 
def visualize_individual(name, data, bound): 
    %matplotlib notebook
    length = len(data)
    
    x = range(0,length)
    y = data

    fig, ax = plt.subplots()
    ax.plot(x, y)

    ax.set(xlabel='Time steps', ylabel='Volume',title= name + " "+ bound + "Traffic")
    ax.grid()

    fig.savefig(name+ "_" + bound + "_traffic.png")

def analyze(data, target_runs, bound):
    %matplotlib inline 
    x = range(0,target_runs)
    x = np.array(x)
    data = np.array(data)
    
    p1 = np.polyfit(x,data,1) #order 1
    p2 = np.polyfit(x,data,2) #order 2
    p3 = np.polyfit(x,data,3) #order 3
   
    
    #plot data and linear fit
    plt.plot(x,data,'o')
    plt.plot(x,np.polyval(p1,x),'r-')
    plt.plot(x,np.polyval(p2,x),'g--')
    plt.plot(x,np.polyval(p3,x),'y:') 
    plt.savefig("Polynomial regression of "+ bound +" traffic data.png")
    plt.show()
    
    print("Slope and intercepts in ascending orders ")
    print(p1)
    print(p2)
    print(p3)

def job():
    #extract data at current time stamp
    list_of_lists = data_extraction()
        
    #save data of individual so far.
    southbound_data.append(list_of_lists[1][individual])
    northbound_data.append(list_of_lists[2][individual])
        
    #visualize overall data.
    visualize_overall(list_of_lists[0],list_of_lists[1],list_of_lists[2],list_of_lists[3])
        
    #track number of runs.
    num_of_runs = (len([file for file in os.listdir('.') if file.endswith('.png')]))
    print("Run number " + str(num_of_runs))
        
    #if target of runs has been reached, visualize individual data.
    if (num_of_runs == target_runs):
        visualize_individual(list_of_lists[0][individual],southbound_data,"Southbound")
        visualize_individual(list_of_lists[0][individual],northbound_data,"Northbound")
        analyze(southbound_data, target_runs,"Southbound")
        analyze(northbound_data, target_runs,"Northbound")
        exit(0) #exit program!
    
schedule.every(0.5).minutes.do(job)
    
while 1:
    schedule.run_pending()
    time.sleep(1) 

In [5]:
#from PyQt5.QtWidgets import QApplication
#from PyQt5.QtCore import QUrl
#from PyQt5.QtWebKit import QWebpage

print("Success")

Success
